In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
# Images Dimensions
img_width, img_height = 128, 128

In [10]:
train_data_dir = 'E:/tes/data/data baru/train'
validation_data_dir = 'E:/tes/data/data baru/validasi'
nb_train_samples = 570
nb_validation_samples = 114
epochs = 50
batch_size = 16

In [77]:
# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

In [5]:
# Build VGG16
model = applications.VGG16(include_top=False, weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [7]:
# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 570 images belonging to 3 classes.


In [43]:
import math

In [46]:
nb_train_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)

In [47]:
print (num_classes)
print (nb_train_samples)

3
570


In [48]:
predict_size_train = int(math.ceil(nb_train_samples / batch_size))

In [69]:
train_features = model.predict_generator(
    train_generator, predict_size_train)
np.save('train_features3.npy', train_features)

In [50]:
# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 114 images belonging to 3 classes.


In [53]:
nb_valid_samples = len(validation_generator.filenames)
num_classesv = len(validation_generator.class_indices)

In [54]:
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

In [70]:
validation_features = model.predict_generator(
    validation_generator, predict_size_validation)
np.save('val_features3.npy', validation_features)

In [71]:
# Build Train Data
train_data = np.load('train_features3.npy')
train_labels = np.array(
    [0] * (nb_train_samples // 3) + [1] * (nb_train_samples // 3) + [2] * (nb_train_samples // 3))


In [72]:
# Build Validation Data
validation_data = np.load('val_features3.npy')
validation_labels = np.array(
    [0] * (nb_validation_samples // 3) + [1] * (nb_validation_samples // 3) + [2] * (nb_validation_samples // 3) )

In [73]:
# Build FC Layer
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(64, activation='relu'))
fc_model.add(Dense(3, activation='softmax'))

In [74]:
print (fc_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                524352    
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 195       
Total params: 524,547
Trainable params: 524,547
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
fc_model.compile(optimizer=adam,
                loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
fc_model.fit(train_data, train_labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels), 
            callbacks=[callbacks])

ValueError: Error when checking target: expected dense_19 to have shape (3,) but got array with shape (1,)